In [ ]:
# ------------------------------------------------------------------
# Example: memory-efficient statistics on a large NetCDF
# ------------------------------------------------------------------
# What “chunks” means:
#   Xarray can read the file in small blocks (chunks) instead of all
#   at once.  Only the blocks you actually need are pulled into RAM,
#   so even a laptop can handle a 30-m grid for all of Ontario. Read
#   more at: docs.xarray.dev/en/stable/generated/xarray.DataArray.chunk.html
# ------------------------------------------------------------------

In [ ]:
#Import xarray (handles NetCDF)
import xarray as xr

In [ ]:
#Open the dataset and tell xarray to use automatic chunk sizes
#    (file must be saved in ../Data/)
ds = xr.open_dataset(
    "../Data/Ontario_30m_Manure_FW_N_P_kg_ha.nc",
    chunks="auto"        # This tells xarray to read in small chunks on demand,
                         # so it never pulls the entire array into RAM at once.
)

In [ ]:
#Choose the variable we care about: total manure (kg/ha/yr)
da = ds["total_manure"]

In [ ]:
#Full-grid statistics (Mask out zeros inside each statistic call (skipna=True))
#Count how many cells are >0 across the entire province
q = da.where(da > 0).quantile([0.25, 0.50, 0.75], skipna=True).compute()
q1, median, q3 = q.values          # 25th, 50th, 75th percentiles

min_nz = da.where(da > 0).min(skipna=True).compute().item()   # non-zero min
max_nz = da.max().compute().item()                            # overall max (>0)

In [ ]:
#Print full-precision results (no rounding)
print("Total Manure – non-zero cells only")
print(f"  Q1 (25th pct) : {q1}")
print(f"  Median        : {median}")
print(f"  Q3 (75th pct) : {q3}")
print(f"  Min (non-zero): {min_nz}")
print(f"  Max           : {max_nz}")